# Job Application Client Notebook

This notebook provides an interactive interface for running the job application process using the CrewAI framework.

In [1]:
import sys
import os

# Add the parent directory to sys.path to be able to import the crew module
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from crewai import Crew
from crew.agents import create_agents
from crew.tasks import create_tasks
from crew.utils import load_config, print_llm_assignments

/home/kedar/miniconda3/envs/rag-pipeline/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/kedar/miniconda3/envs/rag-pipeline/lib/python3.10/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


## Configuration

Set the paths for your resume and configuration file.

In [2]:
resume_path = "../resume.md"
config_path = "config_1.json"

# Load configuration
config = load_config(config_path)

# Check for Serper API key
if 'serper' not in config['api_keys'] or not config['api_keys']['serper']:
    raise ValueError("Serper API key is missing in the config file. Please add it to continue.")

# Print LLM assignments
print_llm_assignments(config)

Set OPENAI_API_KEY environment variable
Set ANTHROPIC_API_KEY environment variable
Set GOOGLE_API_KEY environment variable
Set SERPER_API_KEY environment variable
LLM assignments:
Job Analyzer: openai - gpt-4o-mini
Relevance Selector: openai - gpt-4o-mini
Emphasis Strategist: openai - gpt-4o-mini
Cover Letter Writer: openai - gpt-4o-mini


## Job Information

Enter the job posting URL and optionally, the job description.

In [3]:
job_posting_url = input("Enter the job posting URL: ")
job_description = input("Enter the job description (optional, press Enter to skip): ")

# Add these to the config
config['job_posting_url'] = job_posting_url
config['job_description'] = job_description

## Create Agents and Tasks

In [4]:
# Create agents
job_analyzer, relevance_selector, emphasis_strategist, cover_letter_writer = create_agents(resume_path, config)

# Create tasks
job_analysis_task, relevance_task, emphasis_task, cover_letter_task = create_tasks(
    job_analyzer, relevance_selector, emphasis_strategist, cover_letter_writer
)

## Create and Run the Crew

In [5]:
def run_job_application_process(resume_path, config):
    # Create and run the crew
    job_application_crew = Crew(
        agents=[job_analyzer, relevance_selector, emphasis_strategist, cover_letter_writer],
        tasks=[job_analysis_task, relevance_task, emphasis_task, cover_letter_task],
        verbose=True
    )
    # Prepare inputs for the crew
    job_application_inputs = {
        'job_posting_url': config['job_posting_url'],
        'super_resume_path': resume_path,
        'job_description': config['job_description']
    }

    # Run the crew
    result = job_application_crew.kickoff(inputs=job_application_inputs)
    return result

print("Starting job application process...")
result = run_job_application_process(resume_path, config)

print("\nJob Application Process Completed")
print("\nResults:")
print(result)




Starting job application process...


2024-10-04 17:37:31,411 - 140451987318336 - _common.py-_common:105 - INFO: Backing off send_request(...) for 0.2s (requests.exceptions.ConnectionError: HTTPSConnectionPool(host='us-api.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7fbd86428430>: Failed to resolve 'us-api.i.posthog.com' ([Errno -3] Temporary failure in name resolution)")))


# Agent: Tech Job Researcher and Analyzer
## Task: Analyze the job posting URL provided (https://jobs.lever.co/valence/03c74aad-9aa4-4d7c-823c-2225688ba510) to extract key skills, experiences, and qualifications required. If you cannot scrape the job description or no relevant information is available from the URL, use () which is an alternative but unstructured version of the job posting. Use the tools to gather content and identify and categorize the requirements.


2024-10-04 17:37:31,657 - 140451987318336 - _common.py-_common:105 - INFO: Backing off send_request(...) for 1.3s (requests.exceptions.ConnectionError: HTTPSConnectionPool(host='us-api.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7fbd86d863b0>: Failed to resolve 'us-api.i.posthog.com' ([Errno -3] Temporary failure in name resolution)")))
2024-10-04 17:37:32,996 - 140451987318336 - _common.py-_common:105 - INFO: Backing off send_request(...) for 0.8s (requests.exceptions.ConnectionError: HTTPSConnectionPool(host='us-api.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7fbd86d86ec0>: Failed to resolve 'us-api.i.posthog.com' ([Errno -3] Temporary failure in name resolution)")))
2024-10-04 17:37:33,888 - 140451987318336 - _common.py-_common:120 - ERROR: Giving up send_request(...) after 4 tries 



# Agent: Tech Job Researcher and Analyzer
## Thought: I need to extract the job description from the provided URL to analyze the key skills, experiences, and qualifications required for the job.
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://jobs.lever.co/valence/03c74aad-9aa4-4d7c-823c-2225688ba510\"}"
## Tool Output: 
Valence - Machine Learning EngineerMachine Learning EngineerNew York, New York / Boston, Massachusetts / Philadelphia, Pennsylvania / Washington, DCEngineering /Full-Time (Remote-Friendly) /RemoteApply for this jobValence has built the first-ever, first-to-market AI coaching platform for enterprise, offering personalized, expert, and human-like guidance and support to any leader or employee. At Valence, we're not just talking about the future of work – we're actively shaping it.From your first interaction with us, you'll notice we're different. We're not just offering a job; we're inviting you to help transform the future of work by b

2024-10-04 17:37:48,069 - 140451987318336 - _common.py-_common:105 - INFO: Backing off send_request(...) for 0.4s (requests.exceptions.ConnectionError: HTTPSConnectionPool(host='us-api.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7fbd86429870>: Failed to resolve 'us-api.i.posthog.com' ([Errno -3] Temporary failure in name resolution)")))
2024-10-04 17:37:48,588 - 140451987318336 - _common.py-_common:105 - INFO: Backing off send_request(...) for 1.1s (requests.exceptions.ConnectionError: HTTPSConnectionPool(host='us-api.i.posthog.com', port=443): Max retries exceeded with url: /batch/ (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7fbd86429b10>: Failed to resolve 'us-api.i.posthog.com' ([Errno -3] Temporary failure in name resolution)")))
2024-10-04 17:37:49,711 - 140451987318336 - _common.py-_common:105 - INFO: Backing off send_request(...) for 0.5s (req



# Agent: Relevance Selector
## Final Answer: 
The tailored resume has been constructed to align closely with the Machine Learning Engineer position at Valence, emphasizing relevant skills, experiences, and projects that highlight the candidate's qualifications.


# Agent: Latex Resume Strategist
## Task: Utilize all the data provided by the user and fill in all the data generated in the new resume into the provided LaTeX template. Replace the placeholders marked with << >> with the appropriate information. The [[tag]] markers indicate the beginning and end of repeatable sections. Ensure that bullet points start with strong action verbs and quantify achievements where possible. Bold all keywords that align with the job description to help recruiters quickly analyze the resume. Maintain consistency in style and verb tense throughout the document. Include all relevant certifications, courses, and educational qualifications. Ensure the resume is ATS-friendly by keeping the formatting sim

## Review Output Files

The process should have generated several output files. Let's review them:

In [12]:
output_files = ['resume.md', 'latex_resume.md', 'cover_letter.md']

for file in output_files:
    print(f"\nContents of {file}:")
    print("-" * 40)
    try:
        with open(file, 'r') as f:
            print(f.read())
    except FileNotFoundError:
        print(f"File {file} not found. It may not have been generated.")


Contents of focused_resume.md:
----------------------------------------
File focused_resume.md not found. It may not have been generated.

Contents of emphasize_strategy.md:
----------------------------------------
File emphasize_strategy.md not found. It may not have been generated.

Contents of cover_letter.md:
----------------------------------------
File cover_letter.md not found. It may not have been generated.


## Next Steps

1. Review the generated files and make any necessary adjustments.
2. Use the focused resume and tailored cover letter for your job application.
3. Consider the emphasis strategy when formatting your resume.

Good luck with your job application!